<a href="https://colab.research.google.com/github/bees4ever/ai-workshop-2022/blob/main/YOLO_Workshop_Capgemini_2022_Exercise_02_ready.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crime Detection Workshop
*Capgemini Challenge*

Catch the thief through image analysis and prove your analytical skill!

Estimated Duration: 20 min

Challange Points: 2 

## Story - Spies in the shopping mall 

According to the intelligence service, there was a spy in the shopping mall during regular opening hours. This spy was conspicuously dressed. You are to automatically analyze the images of the surveillance camera for conspicuously dressed persons. The image of the suspicious person is to be handed over to the secret service.
To do this, create the crops of all the people and then extract the three most common colors in the image. If a conspicuous color occurs frequently, it is a suspicious person.


Note: The data are downloaded from kaggle and underly different lizenses, even unkown.

- https://www.kaggle.com/datasets/odins0n/ucf-crime-dataset (CC0 lizense)
- https://www.kaggle.com/datasets/fmena14/crowd-counting (unkown)



Step 1: Dataset
Load the dataset with following command:

In [ ]:
!wget https://github.com/bees4ever/ai-workshop-2022/raw/main/surveillance_dataset/workshop_task_02_dataset.zip

In [ ]:
!unzip workshop_task_02_dataset.zip

## Step 2: Load YOLO

In [ ]:
# as we learned from Exercise 0, we can load the yolo model using pytorch
import torch
model = torch.hub.load('ultralytics/yolov3', 'yolov3')

In [ ]:
# Load YOLO v5
!git clone  'https://github.com/ultralytics/yolov5.git'

In [ ]:
!pip install -qr '/content/yolov5/requirements.txt'  # install dependencies

In [ ]:
!pip install webcolors

## Step 3: Dominant Color



In this section will provide a couple of helper function and do some
pre analysis of this images. You have nothing to do but executing the code
and get introduced with the data source.

In [ ]:
import scipy.cluster
import sklearn.cluster
import numpy
from PIL import Image
import base64
#' some example of explorative data analysis
from IPython.core.display import display, HTML
from google.colab import files


def dominant_colors(image): 
    """Extract the three dominant colors of an image

    :param image:  PIL image input
    :returns: a list of strings representing the three
              dominant colors of an image
    :rtype: list
    """
    image = image.resize((150, 150))      # optional, to reduce time
    ar = numpy.asarray(image)
    shape = ar.shape
    ar = ar.reshape(numpy.product(shape[:2]), shape[2]).astype(float)

    kmeans = sklearn.cluster.MiniBatchKMeans(
        n_clusters=3,
        init="k-means++",
        max_iter=20,
        random_state=1000
    ).fit(ar)
    codes = kmeans.cluster_centers_

    vecs, _dist = scipy.cluster.vq.vq(ar, codes)         # assign codes
    counts, _bins = numpy.histogram(vecs, len(codes))    # count occurrences

    colors = []
    for index in numpy.argsort(counts)[::-1]:
        colors.append(tuple([int(code) for code in codes[index]]))
    return colors   


def imgToHtml(img):
   """convert a image path to a html code using base54

    :param img: path to image
    :returns: html <img> tag including image source
    :rtype: str
   """
   with open(img, 'rb') as binary_file:
     binary_file_data = binary_file.read()
     base64_encoded_data = base64.b64encode(binary_file_data)
     base64_message = base64_encoded_data.decode('utf-8')
     return '<img width="50px" src="data:image/png;base64,' + base64_message + '"/>'



def colorToHtml(color):
    """show colors in nice bars

    :param color: rgb code of color
    :returns: html <div> representation color
    :rtype: str
    """
    return '<div style="display: inline-block; width: 40px; height: 40px; background-color: rgb('+ str(color[0]) +','+ str(color[1] )+',' + str(color[2])+ ');"></div>'


from webcolors import rgb_to_name

from scipy.spatial import KDTree
from webcolors import (
    CSS3_HEX_TO_NAMES,
    hex_to_rgb,
)
#' convert a rgb color code to a name
#' which reduces the three dimension of rgb to
#' one dimension
def convert_rgb_to_names(rgb_tuple):
    """
    convert rgb color code to a readable name
    :param rgb_tuple: tuple rgb color code
    :returns: name of color
    :rtype: str
    """
    # a dictionary of all the hex and their respective names in css3
    css3_db = CSS3_HEX_TO_NAMES
    names = []
    rgb_values = []
    for color_hex, color_name in css3_db.items():
        names.append(color_name)
        rgb_values.append(hex_to_rgb(color_hex))
    
    kdt_db = KDTree(rgb_values)
    distance, index = kdt_db.query(rgb_tuple)
    return names[index]


"""  some constant suspicious_colors, which should be used later to detect
the suspicious person """
#suspicious_colors = ['cornflowerblue',  'palegoldenrod', 'azure', 'lightskyblue', 'lightgoldenrodyellow', 'mediumblue',  'bisque', 'cornsilk',  'slategray', 'darkolivegreen',  'tomato', 'turquoise', 'springgreen', 'mediumaquamarine', 'darkviolet', 'lawngreen', 'lightpink', 'peru', 'mediumslateblue', 'ivory', 'mintcream', 'khaki', 'darkblue', 'darkorange', 'lightseagreen', 'chocolate', 'darkorchid', 'cyan', 'goldenrod', 'lightblue', 'forestgreen', 'darkred', 'mediumvioletred', 'paleturquoise', 'darkcyan', 'lightgray', 'mediumseagreen', 'darkkhaki', 'gainsboro', 'snow', 'peachpuff', 'sienna', 'yellowgreen', 'lightyellow', 'lightsteelblue', 'lemonchiffon', 'lightslategray', 'teal', 'gray', 'palegreen', 'darkslateblue','indigo', 'lightsalmon', 'seagreen', 'mediumpurple', 'papayawhip', 'lightcyan', 'deeppink', 'ghostwhite', 'navajowhite', 'limegreen', 'deepskyblue', 'moccasin', 'lightgreen', 'coral', 'whitesmoke', 'orchid', 'firebrick', 'darksalmon', 'palevioletred', 'mediumspringgreen', 'royalblue', 'magenta', 'honeydew', 'violet', 'salmon', 'pink', 'darkseagreen', 'yellow', 'maroon', 'darkgray', 'saddlebrown', 'midnightblue', 'steelblue', 'skyblue', 'hotpink', 'darkmagenta', 'lightcoral', 'aliceblue', 'darkturquoise', 'floralwhite', 'sandybrown', 'powderblue', 'mediumorchid', 'silver', 'chartreuse', 'antiquewhite', 'plum', 'aquamarine', 'crimson', 'orange', 'indianred', 'oldlace', 'orangered', 'tan', 'mediumturquoise', 'slateblue', 'blueviolet', 'mistyrose', 'seashell', 'olivedrab', 'cadetblue', 'blanchedalmond', 'burlywood',  'darkgreen', 'lavenderblush', 'linen', 'thistle', 'darkgoldenrod', 'dodgerblue', 'greenyellow', 'blue']
suspicious_colors = ['aquamarine',
 'azure',
 'blanchedalmond',
 'blue',
 'blueviolet',
 'chartreuse',
 'coral',
 'cornflowerblue',
 'cornsilk',
 'crimson',
 'cyan',
 'darkblue',
 'darkgoldenrod',
 'darkgreen',
 'darkmagenta',
 'darkorange',
 'darkcyan',
 'darkred',
 'darksalmon',
 'darkseagreen',
 'darkviolet',
 'deeppink',
 'deepskyblue',
 'dodgerblue',
 'firebrick',
 'floralwhite',
 'forestgreen',
 'greenyellow',
 'honeydew',
 'hotpink',
 'indigo',
 'khaki',
 'lavenderblush',
 'lawngreen',
 'lemonchiffon',
 'lightcoral',
 'lightgoldenrodyellow',
 'lightgreen',
 'lightsalmon',
 'lightseagreen',
 'lightskyblue',
 'lightyellow',
 'limegreen',
 'magenta',
 'mediumaquamarine',
 'mediumblue',
 'mediumorchid',
 'mediumpurple',
 'mediumslateblue',
 'mediumspringgreen',
 'mediumturquoise',
 'mediumvioletred',
 'midnightblue',
 'moccasin',
 'navajowhite',
 'oldlace',
 'olivedrab',
 'orange',
 'orangered',
 'orchid',
 'palegoldenrod',
 'palegreen',
 'palevioletred',
 'papayawhip',
 'peachpuff',
 'pink',
 'plum',
 'royalblue',
 'salmon',
 'seagreen',
 'skyblue',
 'slateblue',
 'springgreen',
 'steelblue',
 'tomato',
 'turquoise',
 'violet',
 'yellow',
 'yellowgreen']


## Step 4 - Crop Processing

To actually analyse the person objects, we need to create them, run the following code to extract the crops. 

In [ ]:
#' Run object detection - Extract Them
# Info: This call can ake approx. 10min
!python yolov5/detect.py --source /content/workshop_task_02_dataset  --save-crop

## Step 5: Do the Processing of person objects

This is now your turn. We want you to generate a Data Frame with the columns: *image, seq_number, dominant_color_1, dominant_color_2, dominant_color_3*

For this purpose you simply need to write a function called `append_data_to_df`. Follow the function doc string. You see in the cell after
that this function is called. If the prepared empty data set is filled correctly, your job is done.

In [ ]:
#' TODO: please create here a function to assign the corresponsing data 
# extractions to the Data frame

def append_data_to_df(df_person_dominant_color, img_path, seq_file):
  """ 
  This function appends to the df_person_dominant_color for each avaiable key/
  column the corresponding img_path and seq_number as well as the dominant 
  colors. Ths dominant colors are calculated using `dominant_colors()`.

  :param df_person_dominant_color: dict the raw dict object with the columns
         image, seq_number, dominant_color_1, dominant_color_2, dominant_color_3
  :param img_path: str path to image
  :param seq_file: str image number
  """
  # img_conv can be used as input for `dominant_colors()`.
  img_conv = Image.open(img_path)
  df_person_dominant_color["seq_number"].append(int(re.findall(r"\d+", seq_file)[0]))
  pass
    

In [ ]:
# Here create data frame with persons per sequence and dominator color.
# Run this code to test your function
import re
import os
import pandas as pd
import progressbar as pb
df_person_dominant_color = {"image": [], "seq_number": [], "dominant_color_1": [], "dominant_color_2": [], "dominant_color_3": []}

ext_data_path = 'yolov5/runs/detect/exp/crops/person'

for img_name in pb.progressbar(os.listdir(ext_data_path)):
  main_dir = ext_data_path
  seq_image = os.path.join(main_dir, img_name)
  append_data_to_df(df_person_dominant_color, seq_image, seq_image)
  #for obj_img in os.listdir(person_dir):
    #' TODO: define the function prepared above
    
   


## Step 6: Analyse the results

In [ ]:
# Convert it to real pandas Data frame
df_person_dominant_color = pd.DataFrame(df_person_dominant_color)
#' show a preview of the dataset
df_person_dominant_color.head(30)

In [ ]:
# Run this to see some analysis of the dominant colors of the image
html_str = ''
for (index, row) in df_person_dominant_color.head(10).iterrows():
  html_str += '<b>' + str(row.seq_number) + '</b>' + imgToHtml(row.image) + colorToHtml(row.dominant_color_1) + colorToHtml(row.dominant_color_2) + colorToHtml(row.dominant_color_3)


display(HTML(html_str))

### Result Presentation - Filter for suspicious colors

If your function works, you can run this code to execute the result presentation and the suspicious image will be printed here

In [ ]:
images_found = set()
for (index, row) in pb.progressbar(df_person_dominant_color.iterrows(), max_value=len(df_person_dominant_color)):
  col1 = convert_rgb_to_names(row.dominant_color_1)
  col2 = convert_rgb_to_names(row.dominant_color_2)
  col3 = convert_rgb_to_names(row.dominant_color_3)
  
  if col1 in suspicious_colors or col2 in suspicious_colors or col3 in suspicious_colors :
    print(f'suspicious frame: {row.image} with one of these colors: {col1}, {col2}, {col3}') 
    images_found.add(row.image)

for img in images_found:
  print(img)
  display(HTML(imgToHtml(img)))